In [1]:
!pip install -qU langchain_mistralai langgraph==0.2.19 langchain-community==0.2.16 langchain-openai==0.1.23  python-pptx==0.6.23 python-pptx-interface==0.0.12 python-dotenv==1.0.1 duckduckgo-search==6.2.4
!pip install -qU gigachain==0.2.6 gigachain_community==0.2.6 gigachain-cli==0.0.25 langgraph==0.2.19

In [2]:
import os
import json
import requests
import datetime
from dotenv import load_dotenv
from dateutil import parser

from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationTokenBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models.gigachat import GigaChat
from langchain.agents import create_gigachat_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, SystemMessage

from langchain_core.pydantic_v1 import Field
from langchain.tools import tool

load_dotenv()

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [3]:
class SimpleChatBot:
    def __init__(self, llm, memory_type="buffer", **kwargs):
        self.llm = llm
        memory_classes = {
            "buffer": ConversationBufferMemory,
            "window": ConversationBufferWindowMemory,
            "token": ConversationTokenBufferMemory,
            "summary": ConversationSummaryMemory
        }

        if memory_type in ["token", "summary"]:
            self.memory = memory_classes[memory_type](llm=self.llm, **kwargs)
        else:
            self.memory = memory_classes[memory_type](**kwargs)
        self.conversation = ConversationChain(llm=self.llm,
                                              memory=self.memory)

    def print_memory(self):
        print("\nMemory:", self.conversation.memory.load_memory_variables({}))

    def run(self):
        print("Type 'quit' to end the conversation.")
        while True:
            user_input = input("User: ")
            if user_input.lower() =="quit":
                break
            response = self.conversation.predict(input=user_input)
            print("Assistant:", response)

Now let's check how it works with each type of memory

In [4]:
giga_key = os.environ['SB_AUTH_DATA']

giga = GigaChat(credentials=giga_key,
                model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)
giga.verbose = False

In [ ]:
chat = SimpleChatBot(giga, 'summary')
chat.run()
chat.print_memory()

Type 'quit' to end the conversation.
Assistant: Оба весят одинаково.


Giga generation stopped with reason: blacklist


Assistant: Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.
Assistant: Мы обсуждали разницу в весе между одним килограммом железа и одним килограммом пуха, а также пытались затронуть тему президента мира.

Memory: {'history': 'Человек спрашивает, что тяжелее: один килограмм железа или один килограмм пуха. AI отвечает, что оба весят одинаково. Человека интересует, кто президент мира, но AI не хочет обсуждать эту тему.'}


In [ ]:
chat = SimpleChatBot(giga, 'window', k=1)
chat.run()
chat.print_memory()

Type 'quit' to end the conversation.
Assistant: Оба весят одинаково.


Giga generation stopped with reason: blacklist


Assistant: Не люблю менять тему разговора, но вот сейчас тот самый случай.


Giga generation stopped with reason: blacklist


Assistant: Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?

Memory: {'history': 'Human: О чем мы говорили раньше?\nAI: Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?'}


In [ ]:
chat = SimpleChatBot(giga, 'token', max_token_limit=100)
chat.run()
chat.print_memory()

Type 'quit' to end the conversation.
Assistant: Оба весят одинаково.


Giga generation stopped with reason: blacklist


Assistant: Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?
Assistant: Мы обсуждали разницу в весе между одним килограммом железа и одним килограммом пуха, а также пытались узнать, кто является президентом мира.

Memory: {'history': 'Human: Что тяжелее: один килограмм железа или один килограмм пуха?\nAI: Оба весят одинаково.\nHuman: Кто президент мира?\nAI: Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?\nHuman: О чем мы говорили раньше?\nAI: Мы обсуждали разницу в весе между одним килограммом железа и одним килограммом пуха, а также пытались узнать, кто является президентом мира.'}


In [ ]:
chat = SimpleChatBot(giga, 'buffer')
chat.run()
chat.print_memory()

Type 'quit' to end the conversation.
Assistant: Оба весят одинаково.


Giga generation stopped with reason: blacklist


Assistant: Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.
Assistant: Мы обсуждали, что весит больше: один килограмм железа или один килограмм пуха, и я ответил, что они весят одинаково. Затем вы спросили, кто является президентом мира, и я ответил, что не хочу обсуждать эту тему.

Memory: {'history': 'Human: Что тяжелее: один килограмм железа или один килограмм пуха?\nAI: Оба весят одинаково.\nHuman: Кто президент мира?\nAI: Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.\nHuman: О чем мы говорили ранее?\nAI: Мы обсуждали, что весит больше: один килограмм железа или один килограмм пуха, и я ответил, что они весят одинаково. Затем вы спросили, кто является президентом мира, и я ответил, что не хочу обсуждать эту тему.'}


Report:
- ConversationBufferMemory: сохраняет всю переписку.
- ConversationBufferWindowMemory: хранит только последние несколько сообщений и экономит контекст.
- ConversationTokenBufferMemory: обрезает данные по количеству токенов.
- ConversationSummaryMemory: резюмирует диалог, сохраняя не полную историю, а краткие сводки.

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [5]:
@tool
def get_wheather(
    city: str = Field(description="Name of the city")
    ) -> str:
    """Reports the weather forecast in the city"""
    openweather_key = os.environ.get("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        weather = 'weather description: ' + data['weather'][0]['description'] + ', temperature: ' + str(data['main']['temp'])
        return weather
    except requests.exceptions.RequestException as e:
        return 'RequestExceptions occured'
    except (KeyError, IndexError):
        return 'KeyError or IndexError occured'

In [6]:
class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function
        self.agent = create_gigachat_functions_agent(llm,
                                                     [self.agent_function])
        self.agent_executor = AgentExecutor(agent=self.agent,
                                            tools=[self.agent_function],
                                            verbose=True)

    def run(self, user_input: str):
      result = self.agent_executor.invoke({"input": user_input})
      return result['output']

Let's check it

In [7]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_wheather)
user_input = "Какая погода сейчас в Москве?"
openwheatertool.run(user_input)



> Entering new AgentExecutor chain...

Invoking: `get_wheather` with `{'city': 'Москва'}`


weather description: clear sky, temperature: 5.35В Москве сейчас ясно, температура 5.35°С.

> Finished chain.


'В Москве сейчас ясно, температура 5.35°С.'

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [8]:
def get_geoid(city: str = Field(description="Name of the city")) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)

@tool
def get_url_booking_tickets(
    city_from: str = Field(description="Name of the departure city"),
    city_to: str = Field(description="Name of the city to arrive"),
    date_in_str: str = Field(description="Date of the departure"),
    date_out_str: str = Field(description="Date to return back")
    ) -> str:
    """Reports the url for booking tickets"""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?'
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your url for tickets ordering: {url} from {city_from} to {city_to} on {date_in_str} / {date_out_str}'


@tool
def get_url_booking_hotels(
    date_in_str: str = Field(description="Check-in date"),
    date_out_str: str = Field(description="Eviction date"),
    city: str = Field(description="Name of the hotel's city")
    ) -> str:
    """Reports the url for booking hotels"""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?'
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    return f'Here is your URL for booking: {url} in {city} on {date_in_str} / {date_out_str}'

In [9]:
class MultiAgent:
    def __init__(self, llm, agent_function_wheater, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        self.agent_function_wheater = agent_function_wheater
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

        self.weather_agent = self._create_agent(
            [agent_function_wheater],
            "You are an assistant. Your task is to report ONLY the weather forecast for the city. Ignore other context."
            )
        self.hotel_booking_agent = self._create_agent(
            [agent_function_hotels],
            "You are an assistant. Your task is to send ONLY the url to book a hotel for the specified dates in the city."
            )
        self.ticket_booking_agent = self._create_agent(
            [agent_function_tickets],
            "You are an assistant. Your task is to send ONLY the url to buy tickets from one city to another for the specified dates."
            )

    def _create_agent(self, tools, system_prompt):
        prompt = ChatPromptTemplate.from_messages(
                [
                  ("system", system_prompt),
                  ("human", "{input}"),
                  MessagesPlaceholder("agent_scratchpad"),
                ]
        )
        agent = create_gigachat_functions_agent(
            self.llm,
            tools,
            prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=False)

    def run(self, user_input: str):
        weather_ans = self.weather_agent.invoke({'input': user_input})['output']
        hotels_ans = self.hotel_booking_agent.invoke({'input': user_input})['output']
        tickets_ans = self.ticket_booking_agent.invoke({'input': user_input})['output']
        return hotels_ans + '\n' + tickets_ans + '\n' + weather_ans

In [10]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_wheather, get_url_booking_hotels, get_url_booking_tickets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)

Вот ваша ссылка для бронирования: [отель Санкт-Петербурга с 15 по 24 ноября 2024 года](https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2).
Вот Ваша ссылка для заказа билетов: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-24&toId=c2&when=2024-11-15 от Москвы до Санкт-Петербурга на 15.11.2024 / 24.11.2024
Погода в Санкт-Петербурге с 15 по 24 ноября 2024 года ожидается ясной, температура воздуха составит около 8.36°С.
